In [202]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import numpy as np 
import pandas as pd
import os

In [245]:
train = pd.read_csv('../input/univai/Training Data.csv')
test = pd.read_csv('../input/univai/Test Data.csv')

In [162]:
X = train.iloc[:,:-1]
y = train.iloc[:,-1]

In [163]:
dict1 = {'norent_noown':0,'rented':1,'owned':2}
for i, row in X.iterrows():
    own = X.loc[i,'house_ownership']
    X.at[i,'house_ownership'] = dict1[own]
X['married_'] = pd.get_dummies(X['married'],drop_first=True).values
X['car_ownership_'] = pd.get_dummies(X['car_ownership'],drop_first=True).values
# df2 = pd.get_dummies(X['state'],drop_first=True)
# df3 = pd.get_dummies(X['profession'],drop_first=True)
# X = pd.concat([X,df2],axis=1)

le = preprocessing.LabelEncoder()
X['state_'] = le.fit_transform(X['state'])
# X['city_'] = le.fit_transform(X['city'])
X['profession_'] = le.fit_transform(X['profession'])
X.drop(['Id','married','car_ownership','city','state','profession'],axis=1,inplace=True)

In [171]:
X

,income,age,experience,house_ownership,current_job_years,current_house_years,married_,car_ownership_,state_,profession_
0,1303835,23,3,1,3,13,1,0,13,33
1,7574516,40,10,1,9,13,1,0,14,43
2,3991815,66,4,1,4,10,0,0,12,47
3,6256451,41,2,1,2,12,1,1,17,43
4,5768871,47,11,1,3,14,1,0,22,11
...,...,...,...,...,...,...,...,...,...,...
251995,8154883,43,13,1,6,11,1,0,28,45
251996,2843572,26,10,1,6,11,1,0,13,3
251997,4522448,46,7,1,7,12,1,0,14,17
251998,6507128,45,0,1,0,10,1,0,18,27


In [156]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [203]:
skf = StratifiedKFold(n_splits=10,shuffle=True)
model = LogisticRegression(random_state=0)
model2 = XGBClassifier()

In [205]:
for train_index, test_index in (skf.split(X,y)):
    X_train, X_test, y_train, y_test = X.loc[train_index], X.loc[test_index], y.loc[train_index], y.loc[test_index]
    model2.fit(X_train.values,y_train.values)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:13:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:13:15] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:13:23] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:13:32] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [206]:
preds = model2.predict(X_test.values)

In [207]:
confusion_matrix(y_test,preds), roc_auc_score(y_test,preds), accuracy_score(y_test,preds)

(array([[21819,   281],
        [ 2546,   554]]),
 0.582997372646329,
 0.8878174603174603)

In [126]:
X

,income,age,experience,house_ownership,current_job_years,current_house_years,married_,car_ownership_,city_,profession_
0,1303835,23,3,1,3,13,1,0,251,33
1,7574516,40,10,1,9,13,1,0,227,43
2,3991815,66,4,1,4,10,0,0,8,47
3,6256451,41,2,1,2,12,1,1,54,43
4,5768871,47,11,1,3,14,1,0,296,11
...,...,...,...,...,...,...,...,...,...,...
251995,8154883,43,13,1,6,11,1,0,162,45
251996,2843572,26,10,1,6,11,1,0,251,3
251997,4522448,46,7,1,7,12,1,0,144,17
251998,6507128,45,0,1,0,10,1,0,233,27


In [246]:
X1 = test.copy()

In [247]:
dict1 = {'norent_noown':0,'rented':1,'owned':2}
for i, row in X1.iterrows():
    own = X1.loc[i,'house_ownership']
    X1.at[i,'house_ownership'] = dict1[own]
X1['married_'] = pd.get_dummies(X1['married'],drop_first=True).values
X1['car_ownership_'] = pd.get_dummies(X1['car_ownership'],drop_first=True).values
# df2 = pd.get_dummies(X1['state'],drop_first=True)
# df3 = pd.get_dummies(X1['profession'],drop_first=True)
# X1 = pd.concat([X1,df2],axis=1)

le = preprocessing.LabelEncoder()
X1['state_'] = le.fit_transform(X1['state'])
# X['city_'] = le.fit_transform(X1['city'])
X1['profession_'] = le.fit_transform(X1['profession'])
X1.drop(['id','married','car_ownership','city','state','profession'],axis=1,inplace=True)

In [248]:
X1

,income,age,experience,house_ownership,current_job_years,current_house_years,married_,car_ownership_,state_,profession_
0,7393090,59,19,1,4,13,1,0,28,26
1,1215004,25,5,1,5,10,1,0,14,24
2,8901342,50,12,1,9,14,1,0,14,30
3,1944421,49,9,1,3,12,0,1,14,1
4,13429,25,18,1,13,11,1,1,28,12
...,...,...,...,...,...,...,...,...,...,...
27995,9955481,57,13,1,5,10,1,0,0,44
27996,2917765,47,9,1,9,14,1,0,13,47
27997,8082415,24,5,1,4,13,1,0,14,30
27998,9474180,51,13,1,13,14,1,1,4,7


In [249]:
preds = model2.predict(X1.values)

In [244]:
sub = test[['id']].copy()
sub['risk_flag'] = preds
sub

,id,risk_flag
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
27995,27996,0
27996,27997,0
27997,27998,0
27998,27999,0


In [252]:
sub.to_csv('sub.csv',index=False)